In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from radio_data_utility import RadioV1Dataset as dataset
import argparse
import os
import json
import pickle

from torchvision import transforms

import torch
import numpy as np

from torch.utils.data import DataLoader
import torch.nn as nn

from tqdm import tqdm

import time
import random

In [3]:
with open('./generated_radio_datasets_split/raven_organsmnist_v1_train.pkl', 'rb') as f:
    raven_dataset_v1_organ_train = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v1_val.pkl', 'rb') as f:
    raven_dataset_v1_organ_val = pickle.load(f)

with open('./generated_radio_datasets_split/raven_organsmnist_v1_test.pkl', 'rb') as f:
    raven_dataset_v1_organ_test = pickle.load(f)

In [4]:
from drnet import Solver, train_epoch, validate, count_parameters

In [5]:
RANDOM_SEED = 42

# Set seeds for reproducibility
torch.manual_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

In [6]:
IMG_SIZE = 80

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

eval_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
])

In [7]:
print("Total samples in training set:", len(raven_dataset_v1_organ_train))
print("Total samples in validation set:", len(raven_dataset_v1_organ_val))
print("Total samples in test set:", len(raven_dataset_v1_organ_test))

Total samples in training set: 4400
Total samples in validation set: 550
Total samples in test set: 550


In [8]:
train_dataset = dataset(
    raven_dataset_v1_organ_train, # Use the split training data
    mode="train",
    transform_train=train_transform,
    transform_eval=eval_transform, # Not used by train mode
)

val_dataset = dataset(
    raven_dataset_v1_organ_val, # Use the split validation data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by val mode
    transform_eval=eval_transform,
)

test_dataset = dataset(
    raven_dataset_v1_organ_test, # Use the split test data
    mode="val",    # Set mode to "val" or "test"
    transform_train=train_transform, # Not used by test mode
    transform_eval=eval_transform,
)

In [9]:
import torch
import torch.nn as nn
import numpy as np
import time
import json
from sklearn.model_selection import StratifiedKFold, train_test_split
# ==============================================================================
# 1. SETUP - Constants and Parameters
# ==============================================================================
N_SPLITS = 3 # Number of folds for cross-validation
PATIENCE = 10 # Early stopping patience
VAL_SPLIT_SIZE = 0.20
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EPOCHS = 100
BATCH_SIZE = 128
LEARNING_RATE = 3e-4
BETA1 = 0.9
BETA2 = 0.999
EPSILON = 1e-8

In [ ]:
from types import SimpleNamespace


args = SimpleNamespace(
    model='vit',
    epochs=401,
    load_workers=4,
    dataset_path='',
    save_model_name='',
    img_size=80,
    lr=3e-4,
    beta1=0.9,
    beta2=0.999,
    epsilon=1e-8,
    dataset="raven",
    multi_gpu=False,
    val_every=5,
    test_every=5,
    percent=100,
    # For nargs='+', argparse returns a list, so we mimic that behavior
    trn_configs=['*'],
    tst_configs=['*'],
    silent=False,
    shuffle_first=False,
    check_point=False
)

In [11]:
from torch.utils.data import TensorDataset, ConcatDataset, Subset, DataLoader

dev_dataset = ConcatDataset([train_dataset, val_dataset])
dev_dataset_labels = np.concatenate([train_dataset.labels, val_dataset.labels])

print(f"Total data for Development (Train+Val): {len(dev_dataset)} samples.")
print(f"Test Set size: {len(test_dataset)} samples. (Will not be used until the very end)")

Total data for Development (Train+Val): 4950 samples.
Test Set size: 550 samples. (Will not be used until the very end)


# RADIO-1 STL

In [12]:
model_folder_name = "RADIO1_DRNet"
from tensorboardX import SummaryWriter
import time

In [13]:
from tensorboardX import SummaryWriter
import time

print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []
model_folder_name = "RADIO1_DRNet"  # Folder to save models for each fold

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

    model = Solver(args).to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(BETA1, BETA2), eps=EPSILON)
    criterion = nn.CrossEntropyLoss()
    scheduler = None
    scaler = None

    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        print(f"\nEpoch {epoch}/{EPOCHS} - Fold {fold+1}/{N_SPLITS}")
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION =========================

--- Outer Fold 1/3 ---
Training on 2640, Validating on 660...

Epoch 1/100 - Fold 1/3


Training:   0%|          | 0/21 [00:00<?, ?it/s]/vol/venv/icc24/DGL_ENV/lib/python3.12/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(
Validating: 100%|██████████| 6/6 [00:02<00:00,  2.31it/s]


  New best model for fold 1 at epoch 1: Val Acc: 0.1394

Epoch 2/100 - Fold 1/3


Validating: 100%|██████████| 6/6 [00:02<00:00,  2.33it/s]


  New best model for fold 1 at epoch 2: Val Acc: 0.1424

Epoch 3/100 - Fold 1/3


...


Epoch 14/100 - Fold 3/3


Validating: 100%|██████████| 6/6 [00:02<00:00,  2.31it/s]


  Early stopping at epoch 14.
Loading best model for fold 3 (achieved 0.1470 on inner val set).


Validating: 100%|██████████| 13/13 [00:05<00:00,  2.33it/s]

Performance on Outer Test Set for fold 3: 0.1218

--- NESTED CROSS VALIDATION COMPLETE ---
Cross-validation accuracies on outer folds: ['0.1188', '0.1412', '0.1218']
Mean CV Accuracy: 0.1273 ± 0.0099


In [ ]:
print(f"\n{'='*25} TRAINING FINAL DEPLOYABLE MODEL {'='*25}")

# Create the one-time 90/10 split from the development pool
final_train_indices, final_val_indices = train_test_split(
    np.arange(len(dev_dataset)), test_size=0.1, shuffle=True, stratify=dev_dataset_labels, random_state=RANDOM_SEED
)
final_train_subset = Subset(dev_dataset, final_train_indices)
final_val_subset = Subset(dev_dataset, final_val_indices)
final_train_loader = DataLoader(final_train_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
final_val_loader = DataLoader(final_val_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)


final_model = Solver(args).to(DEVICE)
optimizer = torch.optim.Adam(final_model.parameters(), lr=LEARNING_RATE, betas=(BETA1, BETA2), eps=EPSILON)
criterion = nn.CrossEntropyLoss()
scheduler = None
scaler = None

writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/final_model_training')

best_model_path = f"./saved_models/{model_folder_name}/deployable_model.pth"
best_final_val_acc = 0.0
epochs_without_improvement = 0

print(f"Final training on {len(final_train_subset)} samples, validating on {len(final_val_subset)} for early stopping.")
for epoch in range(1, EPOCHS + 1):
    train_start_time = time.time()
    train_loss, train_acc = train_epoch(final_model, final_train_loader, optimizer, criterion, DEVICE)
    train_end_time = time.time()
    train_time = train_end_time - train_start_time
    
    val_loss, val_acc = validate(final_model, final_val_loader, criterion, DEVICE)
    print(f"Epoch {epoch}: Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")
    
    writer.add_scalar('FinalTrain/Loss/train', train_loss, epoch)
    writer.add_scalar('FinalTrain/Accuracy/train', train_acc, epoch)
    writer.add_scalar('FinalTrain/Loss/val', val_loss, epoch)
    writer.add_scalar('FinalTrain/Accuracy/val', val_acc, epoch)

    writer.add_scalar('FinalTrain/Time/train', train_time, epoch)

    memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
    memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

    writer.add_scalar('FinalTrain/Memory/allocated', memory_allocated, epoch)
    writer.add_scalar('FinalTrain/Memory/reserved', memory_reserved, epoch)

    if val_acc > best_final_val_acc:
        best_final_val_acc = val_acc
        epochs_without_improvement = 0
        torch.save(final_model.state_dict(), best_model_path)
        print(f"  New best model saved with val acc: {best_final_val_acc:.4f}")
    else:
        epochs_without_improvement += 1
        if epochs_without_improvement >= PATIENCE:
            print(f"  Early stopping final training at epoch {epoch}.")
            break
writer.close()
print(f"\n--- FINAL DEPLOYABLE MODEL TRAINING COMPLETE ---")
print(f"Final deployable model saved to '{best_model_path}'")


========================= TRAINING FINAL DEPLOYABLE MODEL =========================
Final training on 4455 samples, validating on 495 for early stopping.


Training:   0%|          | 0/35 [00:00<?, ?it/s]/vol/venv/icc24/DGL_ENV/lib/python3.12/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(
Training: 100%|██████████| 35/35 [00:44<00:00,  1.28s/it]


Epoch Loss: 2.3044, Accuracy: 0.1255


Validating: 100%|██████████| 4/4 [00:02<00:00,  1.87it/s]


Epoch 1: Train Acc: 0.1255 | Val Acc: 0.1131
  New best model saved with val acc: 0.1131


...

Epoch 21: Train Acc: 0.1217 | Val Acc: 0.1354


Training: 100%|██████████| 35/35 [00:44<00:00,  1.27s/it]


Epoch Loss: 2.0827, Accuracy: 0.1279


Validating: 100%|██████████| 4/4 [00:02<00:00,  1.89it/s]

Epoch 22: Train Acc: 0.1279 | Val Acc: 0.1394
  Early stopping final training at epoch 22.

--- FINAL DEPLOYABLE MODEL TRAINING COMPLETE ---
Final deployable model saved to 'RADIO1_DRNet/deployable_model.pth'


In [ ]:
print(f"\n{'='*25} FINAL TEST SET EVALUATION {'='*25}")

best_model_path = f"./saved_models/{model_folder_name}/deployable_model.pth"
criterion = nn.CrossEntropyLoss()

# Load the best model we just created
deployable_model = Solver(args).to(DEVICE)
deployable_model.load_state_dict(torch.load(best_model_path))

# Create a dataloader for the untouched test set
test_start_time = time.time()
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
test_end_time = time.time()
test_time = test_end_time - test_start_time

# Evaluate ONCE
_, final_benchmark_score = validate(deployable_model, test_loader, criterion, DEVICE)

writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/final_benchmark')
# writer.add_hparams(
#     {'model': 'Final Deployable', 'evaluation': 'Official Test Set'},
#     {'hparam/final_accuracy': final_benchmark_score},
#     {'hparam/final_test_time': test_time}
# )

writer.add_scalar('FinalBenchmark/Accuracy/test', final_benchmark_score, 0)
writer.add_scalar('FinalBenchmark/Time/test', test_time, 0)


writer.close()

print(f"\n--- FINAL TEST SET EVALUATION COMPLETE ---")
print(f"The final accuracy of the deployable model on the test set is: {final_benchmark_score:.4f}")


========================= FINAL TEST SET EVALUATION =========================


Validating:   0%|          | 0/5 [00:00<?, ?it/s]/vol/venv/icc24/DGL_ENV/lib/python3.12/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(
Validating: 100%|██████████| 5/5 [00:02<00:00,  1.89it/s]



--- FINAL TEST SET EVALUATION COMPLETE ---
The final accuracy of the deployable model on the test set is: 0.1273


In [ ]:
# ==============================================================================
# FINAL SUMMARY OF ALL RESULTS
# ==============================================================================

print(f"\n\n{'='*30} OVERALL EXPERIMENT SUMMARY {'='*30}")
print(f"1. Nested Cross Validation Performance: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")
print(f"2. Final Deployable Model Performance (on test set): {final_benchmark_score:.4f}")



============================== OVERALL EXPERIMENT SUMMARY ==============================
1. Nested Cross Validation Performance: 0.1279 ± 0.0770
2. Final Deployable Model Performance (on test set): 0.1273


# I-RAVEN to RADIO-1 TL

In [16]:
from tensorboardX import SummaryWriter
import time

# transfer_model_folder_name = "RADIO1_DRNet"
transfer_model_path = "../I_RAVEN/saved_models/best_model_drnet.pth"
model_folder_name = "RADIO1_DRNet_TRANSFER_RAVEN"

print(f"\n{'='*25} PERFORMING NESTED CROSS-VALIDATION TRANSFER {'='*25}")

outer_kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_SEED)
fold_test_results = []

for fold, (train_outer_idx, test_outer_idx) in enumerate(outer_kfold.split(np.zeros(len(dev_dataset)), dev_dataset_labels)):
    print(f"\n--- Outer Fold {fold+1}/{N_SPLITS} ---")
    writer = SummaryWriter(log_dir=f'runs/{model_folder_name}/fold_{fold+1}')

    # Create inner train/val split from the outer training set
    train_outer_labels = dev_dataset_labels[train_outer_idx]
    train_inner_idx, val_inner_idx = train_test_split(
        train_outer_idx, test_size=VAL_SPLIT_SIZE, shuffle=True, stratify=train_outer_labels, random_state=RANDOM_SEED
    )

    # Create subsets from the DEVELOPMENT dataset
    train_inner_subset = Subset(dev_dataset, train_inner_idx)
    val_inner_subset = Subset(dev_dataset, val_inner_idx)
    test_outer_subset = Subset(dev_dataset, test_outer_idx) # This is the "test set" for this fold

    train_loader = DataLoader(train_inner_subset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6, pin_memory=True)
    val_loader = DataLoader(val_inner_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)
    test_loader = DataLoader(test_outer_subset, batch_size=BATCH_SIZE, shuffle=False, num_workers=6, pin_memory=True)

    model = Solver(args).to(DEVICE)

    model.load_state_dict(torch.load(f"{transfer_model_path}"))  # Load the pre-trained model

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(BETA1, BETA2), eps=EPSILON)
    criterion = nn.CrossEntropyLoss()
    scheduler = None
    scaler = None

    best_model_path = f"./saved_models/{model_folder_name}/best_model_fold_{fold+1}.pth"
    os.makedirs(f"./saved_models/{model_folder_name}", exist_ok=True)
    best_val_acc_fold = 0.0
    epochs_without_improvement = 0

    print(f"Training on {len(train_inner_subset)}, Validating on {len(val_inner_subset)}...")
    for epoch in range(1, EPOCHS + 1):
        print(f"\nEpoch {epoch}/{EPOCHS} for Fold {fold+1}")
        train_start_time = time.time()
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
        train_end_time = time.time()
        train_time = train_end_time - train_start_time

        val_loss, val_acc = validate(model, val_loader, criterion, DEVICE)

        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Time: {train_time:.2f}s")

        writer.add_scalar('CV/Loss/train', train_loss, epoch)
        writer.add_scalar('CV/Accuracy/train', train_acc, epoch)
        writer.add_scalar('CV/Loss/val', val_loss, epoch)
        writer.add_scalar('CV/Accuracy/val', val_acc, epoch)
        writer.add_scalar('CV/Time/train', train_time, epoch)

        memory_allocated = torch.cuda.memory_allocated(DEVICE) if torch.cuda.is_available() else 0
        memory_reserved = torch.cuda.memory_reserved(DEVICE) if torch.cuda.is_available() else 0

        writer.add_scalar('CV/Memory/allocated', memory_allocated, epoch)
        writer.add_scalar('CV/Memory/reserved', memory_reserved, epoch)

        if val_acc > best_val_acc_fold:
            best_val_acc_fold = val_acc
            epochs_without_improvement = 0
            torch.save(model.state_dict(), best_model_path)
            print(f"  New best model for fold {fold+1} at epoch {epoch}: Val Acc: {val_acc:.4f}")
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= PATIENCE:
                print(f"  Early stopping at epoch {epoch}.")
                break
    
    # Evaluate the best model for this fold on the outer test set
    print(f"Loading best model for fold {fold+1} (achieved {best_val_acc_fold:.4f} on inner val set).")
    model.load_state_dict(torch.load(best_model_path))
    _, final_fold_test_acc = validate(model, test_loader, criterion, DEVICE)
    print(f"Performance on Outer Test Set for fold {fold+1}: {final_fold_test_acc:.4f}")
    fold_test_results.append(final_fold_test_acc)
    writer.add_scalar('CV/Accuracy/test', final_fold_test_acc, epoch)
    writer.close()

mean_cv_acc = np.mean(fold_test_results)
std_cv_acc = np.std(fold_test_results)
print(f"\n--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---")
print(f"Cross-validation accuracies on outer folds: {[f'{acc:.4f}' for acc in fold_test_results]}")
print(f"Mean CV Accuracy: {mean_cv_acc:.4f} ± {std_cv_acc:.4f}")


========================= PERFORMING NESTED CROSS-VALIDATION TRANSFER =========================

--- Outer Fold 1/3 ---
Training on 2640, Validating on 660...

Epoch 1/100 for Fold 1


Training:   0%|          | 0/21 [00:00<?, ?it/s]/vol/venv/icc24/DGL_ENV/lib/python3.12/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(
Validating: 100%|██████████| 6/6 [00:02<00:00,  2.29it/s]


  Train Loss: 7.2703, Train Acc: 0.1258, Val Loss: 2.0818, Val Acc: 0.1227, Time: 27.17s
  New best model for fold 1 at epoch 1: Val Acc: 0.1227

Epoch 2/100 for Fold 1


Validating: 100%|██████████| 6/6 [00:02<00:00,  2.27it/s]


  Train Loss: 4.4759, Train Acc: 0.1220, Val Loss: 2.0858, Val Acc: 0.1394, Time: 26.31s
  New best model for fold 1 at epoch 2: Val Acc: 0.1394

Epoch 3/100 for Fold 1


...

  Train Loss: 2.4366, Train Acc: 0.1235, Val Loss: 2.0795, Val Acc: 0.1379, Time: 26.72s

Epoch 24/100 for Fold 3


Validating: 100%|██████████| 6/6 [00:02<00:00,  2.25it/s]


  Train Loss: 2.4115, Train Acc: 0.1178, Val Loss: 2.0795, Val Acc: 0.1364, Time: 26.78s
  Early stopping at epoch 24.
Loading best model for fold 3 (achieved 0.1530 on inner val set).


Validating: 100%|██████████| 13/13 [00:05<00:00,  2.30it/s]

Performance on Outer Test Set for fold 3: 0.1327

--- NESTED CROSS VALIDATION TRANSFER COMPLETE ---
Cross-validation accuracies on outer folds: ['0.1297', '0.1364', '0.1327']
Mean CV Accuracy: 0.1329 ± 0.0027
